# Querying DOIs and exporting to CSV



```{admonition} TODO
- [ ] Get section reviewed by a beta reader
- [ ] Add "where next?" links
- [ ] Add some links to helpful prior knowledge:
  - URL format and parameters
  - HTTP requests
  - JSON
  - Pandas
```

This notebook shows how to query the DataCite API for structured metadata about DOIs in bulk, do some processing to transform it into a more useful format, and save the result to a CSV file for further analysis.

## Setting up

First we create a variable, `REPO_ID`, containing the account ID of the Repository we are querying. This can be changed to another Repository account ID (lowercase).

In [1]:
REPO_ID = "rwav.vaavog"

Now we construct a URL for our query. All DataCite API URLs take the form `https://<server>/<endpoint>?parameters...`. Here we use the test server, `api.test.datacite.org`, but for real-world use we would use the production server, `api.datacite.org`. The endpoint for retrieving a list of DOIs is, intuitively, `/dois`. We add the `client_id` parameter to limit the results to only the repository in question.

This operation can potentially return a *lot* of DOIs: there are over 2.4 million registered with DataCite. For that reason the API returns them in batches, referred to as "pages", so each time we receive another page of results we need to check whether there are more still to come. We set the page size to its maximum, 1,000 results in each batch, using `page[size]=1000` and request pagination to be in "cursor" mode with `page[cursor]=1`. The latter ensures we can fetch all available results; otherwise only 10,000 results maximum can be obtained.

- [Find out more about pagination in the DataCite API](https://support.datacite.org/docs/pagination).

_Note: For backward compatibility reasons, the API refers to Repository accounts as "clients"._

In [3]:
url = f"https://api.test.datacite.org/dois?client_id={REPO_ID}&page[size]=1000&page[cursor]=1"

## Running the query

This example uses the Python `requests` package to make it easy to make HTTP requests to the API. We create a HTTP client session: reusing this means there are some options that only need to be set once, and it will also enable some optimisations that we don't need to care about ourselves.

- [Read the full `requests` documentation](https://requests.readthedocs.io/en/latest/)

In [2]:
import requests as rq

session = rq.Session()

Next we run the query by making a HTTP request to this URL. Because of pagination, we need to check each page of results for a `links` attribute containing a `next` link: if this is present we fetch this new URL as well and repeat. The DOI metadata itself is returned in the `data` attribute as a list.

In [4]:
dois = []
while url is not None:
    response = session.get(url)
    response.raise_for_status()
    result = response.json()
    dois += result["data"]
    url = result["links"].get("next", None)

## Processing the results

The results come back to us in JSON format, converted to Python data types (lists, dictionaries, strings and numbers) by the `response.json()` call, and we can use Pandas' `json_normalize` function to conveniently convert this form into a `DataFrame`.

In [5]:
import pandas as pd
from IPython.display import display, FileLink

doi_data = pd.json_normalize(dois)
display(doi_data.tail())

,id,type,attributes.doi,attributes.identifiers,attributes.creators,attributes.titles,attributes.publisher,attributes.publicationYear,attributes.subjects,attributes.contributors,...,attributes.partOfCount,attributes.versionCount,attributes.versionOfCount,attributes.created,attributes.registered,attributes.published,attributes.updated,relationships.client.data.id,relationships.client.data.type,attributes.types.resourceType
82,10.80604/h884-vc25,dois,10.80604/h884-vc25,[],"[{'name': 'Phillip Blair', 'affiliation': [], ...",[{'title': 'Feeling long west people instituti...,University of Poppleton,2021,[],[],...,0,0,0,2023-10-30T15:54:36Z,2023-10-30T15:54:36Z,None,2023-10-30T15:54:38Z,rwav.vaavog,clients,Report
83,10.80604/s790-hg92,dois,10.80604/s790-hg92,[],"[{'name': 'Krystal Galloway', 'affiliation': [...",[{'title': 'Watch across up fill what stop bea...,University of Poppleton,2021,[],[],...,0,0,0,2023-10-30T15:54:36Z,2023-10-30T15:54:36Z,None,2023-10-30T15:54:38Z,rwav.vaavog,clients,Report
84,10.80604/shd5-rx62,dois,10.80604/shd5-rx62,[],"[{'name': 'Allison Stevenson', 'affiliation': ...",[{'title': 'Place start strategy play exist'}],University of Poppleton,2021,[],[],...,0,0,0,2023-10-30T15:54:36Z,2023-10-30T15:54:36Z,None,2023-10-30T15:54:38Z,rwav.vaavog,clients,Report
85,10.80604/7r9m-b748,dois,10.80604/7r9m-b748,[],"[{'name': 'Alexa Bates', 'affiliation': [], 'n...",[{'title': 'Fund design safe get offer improve'}],University of Poppleton,2021,[],[],...,0,0,0,2023-10-30T15:54:37Z,2023-10-30T15:54:37Z,None,2023-10-30T15:54:39Z,rwav.vaavog,clients,Report
86,10.80604/kpzg-f562,dois,10.80604/kpzg-f562,[],"[{'name': 'Amber Allen', 'affiliation': [], 'n...",[{'title': 'Hour sea goal structure least'}],University of Poppleton,2021,[],[],...,0,0,0,2023-10-30T15:54:37Z,2023-10-30T15:54:37Z,None,2023-10-30T15:54:38Z,rwav.vaavog,clients,Report


By default, the DOIs shown in this notebook are test DOIs generated programmatically using [this method](./make-test-dois).

In the "attributes.titles" column, we can see that rather than a single title we have a list of them, which allows for translations of the title into multiple languages to be present. For convenience, we will take only the first title for each item, and extract those into a new "title" column:

In [6]:
def extract_first_title(row):
    all_titles = row['attributes.titles']
    if len(all_titles) == 0:
        return ''
    return all_titles[0]['title']

doi_data['title'] = doi_data.apply(extract_first_title, axis='columns')

Similarly, we will take only the names of the creators, and join them together into a single string with semicolons (`;`):

In [7]:
def extract_creators(row):
    all_creators = row['attributes.creators']
    if len(all_creators) == 0:
        return ''
    return "; ".join(creator["name"] for creator in all_creators)

doi_data['creators'] = doi_data.apply(extract_creators, axis='columns')

Now we can select just the subset of columns we are interested in, and rename them for consistency:

In [8]:
doi_final = doi_data.drop(columns=['attributes.titles', 'attributes.creators']) \
    .rename(columns=lambda x: x.removeprefix('attributes.'))
display(doi_final.tail())

,id,type,doi,identifiers,publisher,publicationYear,subjects,contributors,dates,language,...,versionOfCount,created,registered,published,updated,relationships.client.data.id,relationships.client.data.type,types.resourceType,title,creators
82,10.80604/h884-vc25,dois,10.80604/h884-vc25,[],University of Poppleton,2021,[],[],[],None,...,0,2023-10-30T15:54:36Z,2023-10-30T15:54:36Z,None,2023-10-30T15:54:38Z,rwav.vaavog,clients,Report,Feeling long west people institution near,Phillip Blair; Yvette Macias; Steven Walker
83,10.80604/s790-hg92,dois,10.80604/s790-hg92,[],University of Poppleton,2021,[],[],[],None,...,0,2023-10-30T15:54:36Z,2023-10-30T15:54:36Z,None,2023-10-30T15:54:38Z,rwav.vaavog,clients,Report,Watch across up fill what stop beautiful mission,Krystal Galloway; Jennifer James; Adriana Hunt
84,10.80604/shd5-rx62,dois,10.80604/shd5-rx62,[],University of Poppleton,2021,[],[],[],None,...,0,2023-10-30T15:54:36Z,2023-10-30T15:54:36Z,None,2023-10-30T15:54:38Z,rwav.vaavog,clients,Report,Place start strategy play exist,Allison Stevenson; Sean Hays; Monica Cabrera
85,10.80604/7r9m-b748,dois,10.80604/7r9m-b748,[],University of Poppleton,2021,[],[],[],None,...,0,2023-10-30T15:54:37Z,2023-10-30T15:54:37Z,None,2023-10-30T15:54:39Z,rwav.vaavog,clients,Report,Fund design safe get offer improve,Alexa Bates; Tammy Curry; Virginia Lee
86,10.80604/kpzg-f562,dois,10.80604/kpzg-f562,[],University of Poppleton,2021,[],[],[],None,...,0,2023-10-30T15:54:37Z,2023-10-30T15:54:37Z,None,2023-10-30T15:54:38Z,rwav.vaavog,clients,Report,Hour sea goal structure least,Amber Allen; Darren Green; Robert Gray


Finally we save the resulting data to a CSV file named for today's date.

In [9]:
import datetime as dt

today = dt.datetime.now()
filename = f'{REPO_ID}-{today:%Y%m%d}.csv'
doi_final.to_csv(filename)

FileLink(filename)

/home/jez/Private/Work/DataCite/Book/bl_datacite_api/notebooks/rwav.vaavog-20240815.csv

## More information

This method is highly flexible and allows you to apply specific filters and sort the results, or remove any filters to obtain every last DataCite DOI (this will take a long time!). The above uses a single filter as a URL parameter: `client_id=rwav.vaavog` to fetch only DOIs from a single repository, but we can also filter [by member, consortium, registration date, schema version or several other parameters](https://support.datacite.org/docs/api-queries#applying-filter-parameters). For example, this URL will limit results to DOIs registered in 2022 (pagination options omitted for clarity, see above for details):

```
https://api.datacite.org/dois?registered=2022
```

However, there is also a `query` parameter allowing us to use a very powerful search syntax. For example, this URL returns DOIs where the publisher is DataCite and the resource type is Text or Dataset:

```
https://api.datacite.org/dois?query=publisher:DataCite%20AND%20types.resourceTypeGeneral:(Text%20OR%20Dataset)
```

This query syntax is exactly the same as that used on [DataCite Commons](https://commons.datacite.org) for searching: the previous query can be [run on DataCite Commons](https://commons.datacite.org/?query=publisher:DataCite%20AND%20types.resourceTypeGeneral:(Text%20OR%20Dataset)) to view the results in human-readable format. This makes Commons very useful for constructing and testing queries before using them in the API. When you have a query that works, you can copy the `query=` parameter from your browser's address bar and use it with `https://api.datacite.org/dois` as shown above.